In [29]:
import os
import glob
import pandas as pd

def process_and_merge_data(raw_folder='raw_files', output_folder='data/level1_processing'):
    """
    Merges CSV files from a specified raw data folder based on keywords
    and saves them into a designated output folder.

    This function assumes it is run from the root directory where the 
    'raw_files' folder is located.

    Args:
        raw_folder (str): The name of the folder containing the raw CSV files.
        output_folder (str): The name of the folder where the merged CSV files will be saved.
    """

    # --- 1. Create the output directory if it doesn't exist ---
    # The `exist_ok=True` argument prevents an error if the directory already exists.
    try:
        os.makedirs(output_folder, exist_ok=True)
        print(f"Output directory '{output_folder}' is ready.")
    except OSError as e:
        print(f"Error creating directory {output_folder}: {e}")
        return # Exit the function if directory creation fails

    # --- 2. Define the keywords to search for in filenames ---
    keywords = ['biometric', 'demographic', 'enrolment']

    # --- 3. Loop through each keyword to process files ---
    for keyword in keywords:
        # Construct a search pattern to find all files for the current keyword
        search_pattern = os.path.join(raw_folder, f'*api_data_aadhar_{keyword}*.csv')
        
        # Use glob to find all file paths that match the pattern
        files_to_merge = glob.glob(search_pattern)

        # Check if any files were found
        if not files_to_merge:
            print(f"Warning: No files found for keyword '{keyword}' in '{raw_folder}'.")
            continue # Skip to the next keyword

        print(f"Found {len(files_to_merge)} files for '{keyword}'. Merging now...")

        # --- 4. Read and combine the found CSV files ---
        list_of_dataframes = [pd.read_csv(file) for file in files_to_merge]
        
        if list_of_dataframes:
            # Concatenate all dataframes into a single one
            merged_df = pd.concat(list_of_dataframes, ignore_index=True)

            # --- 5. Save the merged dataframe to the output folder ---
            output_filename = os.path.join(output_folder, f'processed_{keyword}_data.csv')
            merged_df.to_csv(output_filename, index=False)
            print(f"Successfully merged and saved to '{output_filename}'\n")

# --- Example of how to run the function ---
# To run this, make sure you have:
# 1. This script in your project's root folder.
# 2. A folder named 'raw_files' in the same root folder, containing your CSVs.

process_and_merge_data()

Output directory 'data/level1_processing' is ready.
Found 4 files for 'biometric'. Merging now...
Successfully merged and saved to 'data/level1_processing\processed_biometric_data.csv'

Found 5 files for 'demographic'. Merging now...
Successfully merged and saved to 'data/level1_processing\processed_demographic_data.csv'

Found 3 files for 'enrolment'. Merging now...
Successfully merged and saved to 'data/level1_processing\processed_enrolment_data.csv'



In [31]:
import pandas as pd
import os

def find_unique_states():
    """
    Reads multiple processed CSV files, extracts unique state names,
    and saves them to a text file.
    """
    # Define the directory and the list of files to process
    data_dir = os.path.join('data', 'level1_processing')
    files_to_process = [
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv',
        'processed_biometric_data.csv'
    ]

    # A set is used to automatically store unique state names
    unique_states = set()

    # List of possible column names for the state field
    # (to make the script more robust)
    possible_state_columns = ['state', 'State', 'state_name', 'State_Name']

    print("Starting the process of finding unique states...")

    # Loop through each file
    for filename in files_to_process:
        file_path = os.path.join(data_dir, filename)

        try:
            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(file_path)
            print(f"Successfully read {filename}.")

            # Identify the correct state column in the DataFrame
            state_column = None
            for col in possible_state_columns:
                if col in df.columns:
                    state_column = col
                    break
            
            if state_column:
                # Drop any missing values and get the unique states
                found_states = df[state_column].dropna().unique()
                # Add the found states to our set
                unique_states.update(found_states)
                print(f"  Found {len(found_states)} unique state entries in '{state_column}' column.")
            else:
                print(f"  Warning: No state column found in {filename}.")

        except FileNotFoundError:
            print(f"Error: The file {filename} was not found in {data_dir}.")
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

    # Check if any states were found
    if not unique_states:
        print("No state information was found in any of the files.")
        return

    # Sort the states alphabetically
    sorted_states = sorted(list(unique_states))

    # Define the output file path
    output_filename = 'unique_states.txt'

    try:
        # Write the sorted list of states to the output file.
        with open(output_filename, 'w') as f:
            for state in sorted_states:
                f.write(f"{state}\n")
        
        print(f"\nProcess complete. Found {len(sorted_states)} unique states.")
        print(f"The list of unique states has been saved to '{output_filename}'.")

    except Exception as e:
        print(f"An error occurred while writing to the output file: {e}")

if __name__ == "__main__":
    # To run this script, save it as a Python file (e.g., `process_states.py`)
    # in the 'UIDAI HACKATHON' root directory and run it from your terminal.
    find_unique_states()

Starting the process of finding unique states...
Successfully read processed_demographic_data.csv.
  Found 65 unique state entries in 'state' column.
Successfully read processed_enrolment_data.csv.
  Found 55 unique state entries in 'state' column.
Successfully read processed_biometric_data.csv.
  Found 57 unique state entries in 'state' column.

Process complete. Found 68 unique states.
The list of unique states has been saved to 'unique_states.txt'.


In [32]:
import pandas as pd
import os

def clean_and_process_files():
    """
    Cleans the state column in level-1 CSV files by mapping incorrect names
    to official names, removing non-state rows, and saves the cleaned files
    to a level-2 directory.
    """
    # Define source and destination directories
    level1_dir = os.path.join('data', 'level1_processing')
    level2_dir = os.path.join('data', 'level2_processing')

    # Create the level-2 directory if it doesn't exist
    try:
        os.makedirs(level2_dir, exist_ok=True)
        print(f"Directory '{level2_dir}' is ready.")
    except OSError as e:
        print(f"Error creating directory {level2_dir}: {e}")
        return

    # List of files to process from the image
    files_to_process = [
        'processed_biometric_data.csv',
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv'
    ]

    # --- Data Cleaning Definitions ---

    # 1. Define the mapping for all variations to the official state name.
    #    Keys are lowercase for case-insensitive matching.
    state_mapping = {
        'andaman & nicobar islands': 'Andaman and Nicobar Islands',
        'andaman and nicobar islands': 'Andaman and Nicobar Islands',
        'andhra pradesh': 'Andhra Pradesh',
        'arunachal pradesh': 'Arunachal Pradesh',
        'assam': 'Assam',
        'bihar': 'Bihar',
        'chandigarh': 'Chandigarh',
        'chhatisgarh': 'Chhattisgarh',
        'chhattisgarh': 'Chhattisgarh',
        'dadra & nagar haveli': 'Dadra and Nagar Haveli and Daman and Diu',
        'dadra and nagar haveli': 'Dadra and Nagar Haveli and Daman and Diu',
        'daman & diu': 'Dadra and Nagar Haveli and Daman and Diu',
        'daman and diu': 'Dadra and Nagar Haveli and Daman and Diu',
        'dadra and nagar haveli and daman and diu': 'Dadra and Nagar Haveli and Daman and Diu',
        'the dadra and nagar haveli and daman and diu': 'Dadra and Nagar Haveli and Daman and Diu',
        'delhi': 'Delhi',
        'goa': 'Goa',
        'gujarat': 'Gujarat',
        'haryana': 'Haryana',
        'himachal pradesh': 'Himachal Pradesh',
        'jammu & kashmir': 'Jammu and Kashmir',
        'jammu and kashmir': 'Jammu and Kashmir',
        'jharkhand': 'Jharkhand',
        'karnataka': 'Karnataka',
        'kerala': 'Kerala',
        'ladakh': 'Ladakh',
        'lakshadweep': 'Lakshadweep',
        'madhya pradesh': 'Madhya Pradesh',
        'maharashtra': 'Maharashtra',
        'manipur': 'Manipur',
        'meghalaya': 'Meghalaya',
        'mizoram': 'Mizoram',
        'nagaland': 'Nagaland',
        'odisha': 'Odisha',
        'orissa': 'Odisha',
        'pondicherry': 'Puducherry',
        'puducherry': 'Puducherry',
        'punjab': 'Punjab',
        'rajasthan': 'Rajasthan',
        'sikkim': 'Sikkim',
        'tamil nadu': 'Tamil Nadu',
        'tamilnadu': 'Tamil Nadu',
        'telangana': 'Telangana',
        'tripura': 'Tripura',
        'uttar pradesh': 'Uttar Pradesh',
        'uttarakhand': 'Uttarakhand',
        'uttaranchal': 'Uttarakhand',
        'west bengal': 'West Bengal',
        'west bangal': 'West Bengal',
        'westbengal': 'West Bengal',
        'west bengli': 'West Bengal',
        'west  bengal': 'West Bengal' # Handles extra spaces
    }

    # 2. Define non-state entries to be removed.
    #    Using a set for efficient lookup. Stored in lowercase.
    non_states_to_remove = {
        '100000',
        'balanagar',
        'darbhanga',
        'jaipur',
        'madanapalle',
        'nagpur',
        'puttenahalli',
        'raja annamalai puram'
    }

    # --- Processing Loop ---
    print("\nStarting level-1 data cleaning process...")

    for filename in files_to_process:
        input_path = os.path.join(level1_dir, filename)
        output_path = os.path.join(level2_dir, filename)

        try:
            df = pd.read_csv(input_path)
            print(f"\nProcessing '{filename}'...")

            # Find the state column (robustly checks for common names)
            state_column = None
            for col in ['state', 'State', 'state_name', 'State_Name']:
                if col in df.columns:
                    state_column = col
                    break
            
            if not state_column:
                print(f"  Warning: No state column found in '{filename}'. Skipping cleaning for this file.")
                # Optionally, you can copy the file as-is to level2 directory
                # shutil.copy(input_path, output_path)
                continue

            print(f"  Found state data in column: '{state_column}'.")
            original_rows = len(df)

            # Create a standardized, lowercase version of the state column for matching
            # .astype(str) handles potential non-string data. .strip() removes whitespace.
            df['state_lower'] = df[state_column].astype(str).str.lower().str.strip()
            
            # First, remove rows that are not states
            df = df[~df['state_lower'].isin(non_states_to_remove)]
            rows_after_removal = len(df)
            print(f"  Removed {original_rows - rows_after_removal} non-state rows.")

            # Apply the mapping to correct state names
            df[state_column] = df['state_lower'].map(state_mapping)

            # Drop the temporary lowercase column
            df = df.drop(columns=['state_lower'])
            
            # Remove any rows that didn't match a state in our mapping
            df = df.dropna(subset=[state_column])
            rows_after_mapping = len(df)
            print(f"  Removed {rows_after_removal - rows_after_mapping} rows with unmapped state names.")
            
            # Save the cleaned DataFrame to the level-2 directory
            df.to_csv(output_path, index=False)
            print(f"  Successfully cleaned and saved to '{output_path}'. Final row count: {len(df)}.")

        except FileNotFoundError:
            print(f"Error: The file '{filename}' was not found in '{level1_dir}'.")
        except Exception as e:
            print(f"An unexpected error occurred while processing '{filename}': {e}")
    
    print("\n--- Data cleaning process complete. ---")

if __name__ == "__main__":
    clean_and_process_files()

Directory 'data\level2_processing' is ready.

Starting level-1 data cleaning process...

Processing 'processed_biometric_data.csv'...
  Found state data in column: 'state'.
  Removed 0 non-state rows.
  Removed 0 rows with unmapped state names.
  Successfully cleaned and saved to 'data\level2_processing\processed_biometric_data.csv'. Final row count: 1861108.

Processing 'processed_demographic_data.csv'...
  Found state data in column: 'state'.
  Removed 13 non-state rows.
  Removed 0 rows with unmapped state names.
  Successfully cleaned and saved to 'data\level2_processing\processed_demographic_data.csv'. Final row count: 2071687.

Processing 'processed_enrolment_data.csv'...
  Found state data in column: 'state'.
  Removed 22 non-state rows.
  Removed 0 rows with unmapped state names.
  Successfully cleaned and saved to 'data\level2_processing\processed_enrolment_data.csv'. Final row count: 1006007.

--- Data cleaning process complete. ---


In [33]:
import pandas as pd
import os

def check_unique_states_level2():
    """
    Reads the cleaned level-2 CSV files, extracts all unique state names,
    and prints the consolidated list to the console for verification.
    """
    # Define the directory where the cleaned files are located
    level2_dir = os.path.join('data', 'level2_processing')

    # Check if the directory exists before proceeding
    if not os.path.isdir(level2_dir):
        print(f"Error: The directory '{level2_dir}' was not found.")
        print("Please run the cleaning script first to generate the level-2 files.")
        return

    # List of files to process
    files_to_process = [
        'processed_biometric_data.csv',
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv'
    ]

    # Use a set to automatically store unique state names
    unique_states = set()

    # List of possible column names for the state field
    possible_state_columns = ['state', 'State', 'state_name', 'State_Name']

    print(f"--- Checking for unique states in '{level2_dir}' ---")

    # Loop through each file
    for filename in files_to_process:
        file_path = os.path.join(level2_dir, filename)

        try:
            # Read the CSV file
            df = pd.read_csv(file_path)
            print(f"Reading '{filename}'...")

            # Identify the state column in the DataFrame
            state_column = None
            for col in possible_state_columns:
                if col in df.columns:
                    state_column = col
                    break
            
            if state_column:
                # Get the unique, non-null states from the column
                found_states = df[state_column].dropna().unique()
                # Add the found states to our master set
                unique_states.update(found_states)
                print(f"  Found {len(found_states)} unique state entries in this file.")
            else:
                print(f"  Warning: No state column found in {filename}.")

        except FileNotFoundError:
            print(f"  Error: The file '{filename}' was not found. Skipping.")
        except Exception as e:
            print(f"  An error occurred while processing {filename}: {e}")

    # --- Display the final results ---
    if not unique_states:
        print("\nNo state information could be found in the level-2 files.")
        return

    # Convert the set to a sorted list for clean display
    sorted_states = sorted(list(unique_states))

    print("\n--- Verification Complete ---")
    print(f"Total number of unique states and UTs found: {len(sorted_states)}")
    print("---------------------------------")
    
    # Print each state name
    for state in sorted_states:
        print(state)
    
    print("---------------------------------")


check_unique_states_level2()

--- Checking for unique states in 'data\level2_processing' ---
Reading 'processed_biometric_data.csv'...
  Found 36 unique state entries in this file.
Reading 'processed_demographic_data.csv'...
  Found 36 unique state entries in this file.
Reading 'processed_enrolment_data.csv'...
  Found 36 unique state entries in this file.

--- Verification Complete ---
Total number of unique states and UTs found: 36
---------------------------------
Andaman and Nicobar Islands
Andhra Pradesh
Arunachal Pradesh
Assam
Bihar
Chandigarh
Chhattisgarh
Dadra and Nagar Haveli and Daman and Diu
Delhi
Goa
Gujarat
Haryana
Himachal Pradesh
Jammu and Kashmir
Jharkhand
Karnataka
Kerala
Ladakh
Lakshadweep
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Nagaland
Odisha
Puducherry
Punjab
Rajasthan
Sikkim
Tamil Nadu
Telangana
Tripura
Uttar Pradesh
Uttarakhand
West Bengal
---------------------------------


In [34]:
import pandas as pd
import os
import re

def extract_districts_by_state():
    """
    Reads cleaned level-2 data, groups by state, and saves a unique,
    sorted list of districts for each state into a separate text file.
    """
    # Define source and destination directories
    level2_dir = os.path.join('data', 'level2_processing')
    output_dir = os.path.join('state_wise_districts')

    # --- 1. Setup and Pre-checks ---

    # Check if the source directory exists
    if not os.path.isdir(level2_dir):
        print(f"Error: The source directory '{level2_dir}' was not found.")
        print("Please ensure you have run the previous cleaning script to generate the level-2 files.")
        return

    # Create the output directory if it doesn't exist
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Output will be saved in '{output_dir}' directory.")
    except OSError as e:
        print(f"Error: Could not create directory {output_dir}: {e}")
        return

    # --- 2. Read and Combine all Level-2 Data ---
    
    files_to_process = [
        'processed_biometric_data.csv',
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv'
    ]
    
    all_dataframes = []
    print("\nReading and combining level-2 files...")

    for filename in files_to_process:
        file_path = os.path.join(level2_dir, filename)
        try:
            df = pd.read_csv(file_path)
            all_dataframes.append(df)
            print(f"  Successfully loaded '{filename}'.")
        except FileNotFoundError:
            print(f"  Warning: File not found: '{filename}'. Skipping.")
        except Exception as e:
            print(f"  An error occurred reading {filename}: {e}")

    if not all_dataframes:
        print("\nError: No data could be loaded. Aborting process.")
        return

    # Combine all loaded data into a single DataFrame
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"Combined data contains {len(combined_df)} total rows.")

    # --- 3. Identify Columns and Clean Data ---

    # Find the state and district columns robustly
    state_col = next((col for col in ['state', 'State'] if col in combined_df.columns), None)
    district_col = next((col for col in ['district', 'District'] if col in combined_df.columns), None)

    if not state_col or not district_col:
        print("\nError: Could not find 'state' or 'district' columns in the combined data. Aborting.")
        return
    
    print(f"Using '{state_col}' for states and '{district_col}' for districts.")

    # Drop rows where state or district is missing
    original_rows = len(combined_df)
    combined_df.dropna(subset=[state_col, district_col], inplace=True)
    print(f"Removed {original_rows - len(combined_df)} rows with missing state or district info.")
    
    # --- 4. Group by State and Extract Districts ---
    
    print("\nProcessing data for each state...")
    # Group the DataFrame by the state column
    grouped_by_state = combined_df.groupby(state_col)
    
    states_processed_count = 0
    for state_name, group_df in grouped_by_state:
        # Get unique district names and sort them alphabetically
        unique_districts = sorted(group_df[district_col].astype(str).unique())

        # Create a safe and valid filename for the text file
        # Replaces spaces with underscores and removes other invalid characters
        safe_filename = re.sub(r'[^a-zA-Z0-9_]', '', state_name.replace(' ', '_').replace('&', 'and'))
        output_filename = f"{safe_filename}_districts.txt"
        output_path = os.path.join(output_dir, output_filename)

        try:
            # Write the list of districts to the new text file
            with open(output_path, 'w') as f:
                for district in unique_districts:
                    f.write(f"{district}\n")
            
            print(f"  -> Saved {len(unique_districts)} unique districts for '{state_name}' to '{output_filename}'")
            states_processed_count += 1
        except Exception as e:
            print(f"  -> Error writing file for '{state_name}': {e}")
            
    print(f"\n--- Process Complete ---")
    print(f"Successfully processed and created files for {states_processed_count} states/UTs.")

if __name__ == "__main__":
    extract_districts_by_state()

Output will be saved in 'state_wise_districts' directory.

Reading and combining level-2 files...
  Successfully loaded 'processed_biometric_data.csv'.
  Successfully loaded 'processed_demographic_data.csv'.
  Successfully loaded 'processed_enrolment_data.csv'.
Combined data contains 4938802 total rows.
Using 'state' for states and 'district' for districts.
Removed 0 rows with missing state or district info.

Processing data for each state...
  -> Saved 5 unique districts for 'Andaman and Nicobar Islands' to 'Andaman_and_Nicobar_Islands_districts.txt'
  -> Saved 49 unique districts for 'Andhra Pradesh' to 'Andhra_Pradesh_districts.txt'
  -> Saved 25 unique districts for 'Arunachal Pradesh' to 'Arunachal_Pradesh_districts.txt'
  -> Saved 38 unique districts for 'Assam' to 'Assam_districts.txt'
  -> Saved 48 unique districts for 'Bihar' to 'Bihar_districts.txt'
  -> Saved 3 unique districts for 'Chandigarh' to 'Chandigarh_districts.txt'
  -> Saved 41 unique districts for 'Chhattisgarh' t

In [35]:
import json
import os
from collections import defaultdict

def create_state_district_list_with_unknowns():
    """
    Finds a GeoJSON file, extracts state and district names, and groups
    any features with missing data under an 'Unknown' category instead of skipping them.
    """
    data_dir = 'data'
    output_filename = os.path.join('state_districts_list.json')
    
    # --- 1. Find the GeoJSON file ---
    geojson_file_path = None
    try:
        for filename in os.listdir(data_dir):
            if filename.lower().endswith('.geojson'):
                geojson_file_path = os.path.join(data_dir, filename)
                print(f"Found GeoJSON file: '{geojson_file_path}'")
                break
    except FileNotFoundError:
        print(f"Error: The directory '{data_dir}' does not exist.")
        return

    if not geojson_file_path:
        print(f"Error: No .geojson file found in the '{data_dir}' directory.")
        return

    # --- 2. Read the GeoJSON file ---
    try:
        with open(geojson_file_path, 'r') as f:
            geojson_data = json.load(f)
        print("Successfully loaded the GeoJSON data.")
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    # --- 3. Extract States and Districts, Handling Missing Data ---
    
    state_district_map = defaultdict(list)
    print("Extracting state and district names...")

    if 'features' in geojson_data and isinstance(geojson_data['features'], list):
        for feature in geojson_data['features']:
            properties = feature.get('properties')
            
            # Check if the 'properties' object itself is missing or not a dictionary
            if not isinstance(properties, dict):
                # This is a severely malformed feature, group it under Unknown.
                state_district_map['Unknown'].append('Unknown District (No Properties)')
                continue

            # .get() is used to provide a default value if a key is missing.
            # This is the core change to prevent skipping.
            state_name = properties.get('st_nm', 'Unknown')
            district_name = properties.get('district', 'Unknown District (Missing Name)')
            
            # If the state name was missing, we map it to the 'Unknown' key.
            # If the district name was missing, its default value will be added.
            state_district_map[state_name].append(district_name)
    
    # --- 4. Clean up the data ---
    print("Cleaning and sorting the district lists...")
    final_data = {}
    for state, districts in state_district_map.items():
        # Using set() removes duplicates, and sorted() puts them in order.
        final_data[state] = sorted(list(set(districts)))

    # --- 5. Save the result to a new JSON file ---
    try:
        with open(output_filename, 'w') as f:
            json.dump(final_data, f, indent=4)
        print(f"\n--- Process Complete ---")
        print(f"Successfully saved the state-district list to '{output_filename}'.")
        print("Features with missing data have been grouped under the 'Unknown' key.")
    except Exception as e:
        print(f"An error occurred while writing the output file: {e}")

if __name__ == "__main__":
    create_state_district_list_with_unknowns()

Found GeoJSON file: 'data\india_districts_simplified.geojson'
Successfully loaded the GeoJSON data.
Extracting state and district names...
Cleaning and sorting the district lists...

--- Process Complete ---
Successfully saved the state-district list to 'state_districts_list.json'.
Features with missing data have been grouped under the 'Unknown' key.


In [36]:
import pandas as pd
import os
from collections import defaultdict

# This dictionary is the "source of truth". It contains every state and its
# final, officially validated list of districts.
OFFICIAL_DISTRICTS_PER_STATE = {
    'Andaman and Nicobar Islands': ['Nicobars', 'North and Middle Andaman', 'South Andaman'],
    'Andhra Pradesh': ['Alluri Sitharama Raju', 'Anakapalli', 'Ananthapuramu', 'Annamayya', 'Bapatla', 'Chittoor', 'Dr. B.R. Ambedkar Konaseema', 'East Godavari', 'Eluru', 'Guntur', 'Kakinada', 'Krishna', 'Kurnool', 'N.T.R.', 'Nandyal', 'Palnadu', 'Parvathipuram Manyam', 'Prakasam', 'S.P.S. Nellore', 'Sri Sathya Sai', 'Srikakulam', 'Tirupati', 'Visakhapatnam', 'Vizianagaram', 'West Godavari', 'Y.S.R.'],
    'Arunachal Pradesh': ['Anjaw', 'Changlang', 'Dibang Valley', 'East Kameng', 'East Siang', 'Kamle', 'Kra Daadi', 'Kurung Kumey', 'Lepa Rada', 'Lohit', 'Longding', 'Lower Dibang Valley', 'Lower Siang', 'Lower Subansiri', 'Namsai', 'Pakke Kessang', 'Papum Pare', 'Shi Yomi', 'Siang', 'Tawang', 'Tirap', 'Upper Siang', 'Upper Subansiri', 'West Kameng', 'West Siang'],
    'Assam': ['Bajali', 'Baksa', 'Barpeta', 'Biswanath', 'Bongaigaon', 'Cachar', 'Charaideo', 'Chirang', 'Darrang', 'Dhemaji', 'Dhubri', 'Dibrugarh', 'Dima Hasao', 'Goalpara', 'Golaghat', 'Hailakandi', 'Hojai', 'Jorhat', 'Kamrup', 'Kamrup Metropolitan', 'Karbi Anglong', 'Karimganj', 'Kokrajhar', 'Lakhimpur', 'Majuli', 'Morigaon', 'Nagaon', 'Nalbari', 'Sivasagar', 'Sonitpur', 'South Salmara-Mankachar', 'Tamulpur', 'Tinsukia', 'Udalguri', 'West Karbi Anglong'],
    'Bihar': ['Araria', 'Arwal', 'Aurangabad', 'Banka', 'Begusarai', 'Bhagalpur', 'Bhojpur', 'Buxar', 'Darbhanga', 'East Champaran', 'Gaya', 'Gopalganj', 'Jamui', 'Jehanabad', 'Kaimur', 'Katihar', 'Khagaria', 'Kishanganj', 'Lakhisarai', 'Madhepura', 'Madhubani', 'Munger', 'Muzaffarpur', 'Nalanda', 'Nawada', 'Patna', 'Purnia', 'Rohtas', 'Saharsa', 'Samastipur', 'Saran', 'Sheikhpura', 'Sheohar', 'Sitamarhi', 'Siwan', 'Supaul', 'Vaishali', 'West Champaran'],
    'Chandigarh': ['Chandigarh'],
    'Chhattisgarh': ['Balod', 'Baloda Bazar', 'Balrampur', 'Bastar', 'Bemetara', 'Bijapur', 'Bilaspur', 'Dantewada', 'Dhamtari', 'Durg', 'Gariyaband', 'Gaurela-Pendra-Marwahi', 'Janjgir-Champa', 'Jashpur', 'Kabirdham', 'Kanker', 'Khairagarh-Chhuikhadan-Gandai', 'Kondagaon', 'Korba', 'Koriya', 'Mahasamund', 'Manendragarh-Chirmiri-Bharatpur', 'Mohla-Manpur-Ambagarh Chowki', 'Mungeli', 'Narayanpur', 'Raigarh', 'Raipur', 'Rajnandgaon', 'Sakti', 'Sarangarh-Bilaigarh', 'Sukma', 'Surajpur', 'Surguja'],
    'Dadra and Nagar Haveli and Daman and Diu': ['Dadra and Nagar Haveli', 'Daman', 'Diu'],
    'Delhi': ['Central Delhi', 'East Delhi', 'New Delhi', 'North Delhi', 'North East Delhi', 'North West Delhi', 'Shahdara', 'South Delhi', 'South East Delhi', 'South West Delhi', 'West Delhi'],
    'Goa': ['North Goa', 'South Goa'],
    'Gujarat': ['Ahmedabad', 'Amreli', 'Anand', 'Aravalli', 'Banaskantha', 'Bharuch', 'Bhavnagar', 'Botad', 'Chhota Udepur', 'Dahod', 'Dang', 'Devbhoomi Dwarka', 'Gandhinagar', 'Gir Somnath', 'Jamnagar', 'Junagadh', 'Kutch', 'Kheda', 'Mahisagar', 'Mehsana', 'Morbi', 'Narmada', 'Navsari', 'Panchmahal', 'Patan', 'Porbandar', 'Rajkot', 'Sabarkantha', 'Surat', 'Surendranagar', 'Tapi', 'Vadodara', 'Valsad'],
    'Haryana': ['Ambala', 'Bhiwani', 'Charkhi Dadri', 'Faridabad', 'Fatehabad', 'Gurugram', 'Hisar', 'Jhajjar', 'Jind', 'Kaithal', 'Karnal', 'Kurukshetra', 'Mahendragarh', 'Nuh', 'Palwal', 'Panchkula', 'Panipat', 'Rewari', 'Rohtak', 'Sirsa', 'Sonipat', 'Yamunanagar'],
    'Himachal Pradesh': ['Bilaspur', 'Chamba', 'Hamirpur', 'Kangra', 'Kinnaur', 'Kullu', 'Lahaul and Spiti', 'Mandi', 'Shimla', 'Sirmaur', 'Solan', 'Una'],
    'Jammu and Kashmir': ['Anantnag', 'Bandipora', 'Baramulla', 'Budgam', 'Doda', 'Ganderbal', 'Jammu', 'Kathua', 'Kishtwar', 'Kulgam', 'Kupwara', 'Poonch', 'Pulwama', 'Rajouri', 'Ramban', 'Reasi', 'Samba', 'Shopian', 'Srinagar', 'Udhampur'],
    'Jharkhand': ['Bokaro', 'Chatra', 'Deoghar', 'Dhanbad', 'Dumka', 'East Singhbhum', 'Garhwa', 'Giridih', 'Godda', 'Gumla', 'Hazaribagh', 'Jamtara', 'Khunti', 'Koderma', 'Latehar', 'Lohardaga', 'Pakur', 'Palamu', 'Ramgarh', 'Ranchi', 'Sahibganj', 'Seraikela Kharsawan', 'Simdega', 'West Singhbhum'],
    'Karnataka': ['Bagalkot', 'Ballari', 'Belagavi', 'Bengaluru Rural', 'Bengaluru Urban', 'Bidar', 'Chamarajanagar', 'Chikkaballapur', 'Chikkamagaluru', 'Chitradurga', 'Dakshina Kannada', 'Davanagere', 'Dharwad', 'Gadag', 'Hassan', 'Haveri', 'Kalaburagi', 'Kodagu', 'Kolar', 'Koppal', 'Mandya', 'Mysuru', 'Raichur', 'Ramanagara', 'Shivamogga', 'Tumakuru', 'Udupi', 'Uttara Kannada', 'Vijayanagara', 'Vijayapura', 'Yadgir'],
    'Kerala': ['Alappuzha', 'Ernakulam', 'Idukki', 'Kannur', 'Kasaragod', 'Kollam', 'Kottayam', 'Kozhikode', 'Malappuram', 'Palakkad', 'Pathanamthitta', 'Thiruvananthapuram', 'Thrissur', 'Wayanad'],
    'Ladakh': ['Kargil', 'Leh'],
    'Lakshadweep': ['Lakshadweep'],
    'Madhya Pradesh': ['Agar Malwa', 'Alirajpur', 'Anuppur', 'Ashoknagar', 'Balaghat', 'Barwani', 'Betul', 'Bhind', 'Bhopal', 'Burhanpur', 'Chhatarpur', 'Chhindwara', 'Damoh', 'Datia', 'Dewas', 'Dhar', 'Dindori', 'Guna', 'Gwalior', 'Harda', 'Indore', 'Jabalpur', 'Jhabua', 'Katni', 'Khandwa', 'Khargone', 'Maihar', 'Mandla', 'Mandsaur', 'Mauganj', 'Morena', 'Narmadapuram', 'Narsinghpur', 'Neemuch', 'Niwari', 'Pandhurna', 'Panna', 'Raisen', 'Rajgarh', 'Ratlam', 'Rewa', 'Sagar', 'Satna', 'Sehore', 'Seoni', 'Shahdol', 'Shajapur', 'Sheopur', 'Shivpuri', 'Sidhi', 'Singrauli', 'Tikamgarh', 'Ujjain', 'Umaria', 'Vidisha'],
    'Maharashtra': ['Ahmednagar', 'Akola', 'Amravati', 'Beed', 'Bhandara', 'Buldhana', 'Chandrapur', 'Chhatrapati Sambhajinagar', 'Dharashiv', 'Dhule', 'Gadchiroli', 'Gondia', 'Hingoli', 'Jalgaon', 'Jalna', 'Kolhapur', 'Latur', 'Mumbai City', 'Mumbai Suburban', 'Nagpur', 'Nanded', 'Nandurbar', 'Nashik', 'Palghar', 'Parbhani', 'Pune', 'Raigad', 'Ratnagiri', 'Sangli', 'Satara', 'Sindhudurg', 'Solapur', 'Thane', 'Wardha', 'Washim', 'Yavatmal'],
    'Manipur': ['Bishnupur', 'Chandel', 'Churachandpur', 'Imphal East', 'Imphal West', 'Jiribam', 'Kakching', 'Kangpokpi', 'Pherzawl', 'Senapati', 'Tamenglong', 'Thoubal', 'Ukhrul'],
    'Meghalaya': ['East Garo Hills', 'East Jaintia Hills', 'East Khasi Hills', 'Eastern West Khasi Hills', 'North Garo Hills', 'Ri-Bhoi', 'South Garo Hills', 'South West Garo Hills', 'South West Khasi Hills', 'West Garo Hills', 'West Jaintia Hills', 'West Khasi Hills'],
    'Mizoram': ['Aizawl', 'Champhai', 'Hnahthial', 'Khawzawl', 'Kolasib', 'Lawngtlai', 'Lunglei', 'Mamit', 'Saiha', 'Saitual', 'Serchhip'],
    'Nagaland': ['Chümoukedima', 'Dimapur', 'Kiphire', 'Kohima', 'Longleng', 'Mokokchung', 'Mon', 'Niuland', 'Noklak', 'Peren', 'Phek', 'Shamator', 'Tseminyü', 'Tuensang', 'Wokha', 'Zunheboto'],
    'Odisha': ['Angul', 'Balangir', 'Balasore', 'Bargarh', 'Bhadrak', 'Boudh', 'Cuttack', 'Deogarh', 'Dhenkanal', 'Gajapati', 'Ganjam', 'Jagatsinghpur', 'Jajpur', 'Jharsuguda', 'Kalahandi', 'Kandhamal', 'Kendrapara', 'Kendujhar', 'Khordha', 'Koraput', 'Malkangiri', 'Mayurbhanj', 'Nabarangpur', 'Nayagarh', 'Nuapada', 'Puri', 'Rayagada', 'Sambalpur', 'Subarnapur', 'Sundargarh'],
    'Puducherry': ['Karaikal', 'Mahé', 'Puducherry', 'Yanam'],
    'Punjab': ['Amritsar', 'Barnala', 'Bathinda', 'Faridkot', 'Fatehgarh Sahib', 'Fazilka', 'Ferozepur', 'Gurdaspur', 'Hoshiarpur', 'Jalandhar', 'Kapurthala', 'Ludhiana', 'Malerkotla', 'Mansa', 'Moga', 'Pathankot', 'Patiala', 'Rupnagar', 'Sahibzada Ajit Singh Nagar', 'Sangrur', 'Shaheed Bhagat Singh Nagar', 'Sri Muktsar Sahib', 'Tarn Taran'],
    'Rajasthan': ['Ajmer', 'Alwar', 'Balotra', 'Banswara', 'Baran', 'Barmer', 'Beawar', 'Bharatpur', 'Bhilwara', 'Bikaner', 'Bundi', 'Chittorgarh', 'Churu', 'Dausa', 'Deeg', 'Dholpur', 'Didwana-Kuchaman', 'Dungarpur', 'Hanumangarh', 'Jaipur', 'Jaisalmer', 'Jalore', 'Jhalawar', 'Jhunjhunu', 'Jodhpur', 'Karauli', 'Khairthal-Tijara', 'Kota', 'Kotputli-Behror', 'Nagaur', 'Pali', 'Phalodi', 'Pratapgarh', 'Rajsamand', 'Salumbar', 'Sawai Madhopur', 'Sikar', 'Sirohi', 'Sri Ganganagar', 'Tonk', 'Udaipur'],
    'Sikkim': ['Gangtok', 'Gyalshing', 'Mangan', 'Namchi'],
    'Tamil Nadu': ['Ariyalur', 'Chengalpattu', 'Chennai', 'Coimbatore', 'Cuddalore', 'Dharmapuri', 'Dindigul', 'Erode', 'Kallakurichi', 'Kancheepuram', 'Kanniyakumari', 'Karur', 'Krishnagiri', 'Madurai', 'Mayiladuthurai', 'Nagapattinam', 'Namakkal', 'Perambalur', 'Pudukkottai', 'Ramanathapuram', 'Ranipet', 'Salem', 'Sivaganga', 'Tenkasi', 'Thanjavur', 'The Nilgiris', 'Theni', 'Thiruvarur', 'Thoothukudi', 'Tiruchirappalli', 'Tirunelveli', 'Tirupattur', 'Tiruppur', 'Tiruvallur', 'Tiruvannamalai', 'Vellore', 'Viluppuram', 'Virudhunagar'],
    'Telangana': ['Adilabad', 'Bhadradri Kothagudem', 'Hanumakonda', 'Hyderabad', 'Jagitial', 'Jangaon', 'Jayashankar Bhupalpally', 'Jogulamba Gadwal', 'Kamareddy', 'Karimnagar', 'Khammam', 'Komaram Bheem', 'Mahabubabad', 'Mahabubnagar', 'Mancherial', 'Medak', 'Medchal-Malkajgiri', 'Mulugu', 'Nagarkurnool', 'Nalgonda', 'Narayanpet', 'Nirmal', 'Nizamabad', 'Peddapalli', 'Rajanna Sircilla', 'Rangareddy', 'Sangareddy', 'Siddipet', 'Suryapet', 'Vikarabad', 'Wanaparthy', 'Warangal', 'Yadadri Bhuvanagiri'],
    'Tripura': ['Dhalai', 'Gomati', 'Khowai', 'North Tripura', 'Sepahijala', 'South Tripura', 'Unakoti', 'West Tripura'],
    'Uttar Pradesh': ['Agra', 'Aligarh', 'Ambedkar Nagar', 'Amethi', 'Amroha', 'Auraiya', 'Ayodhya', 'Azamgarh', 'Baghpat', 'Bahraich', 'Ballia', 'Balrampur', 'Banda', 'Barabanki', 'Bareilly', 'Basti', 'Bhadohi', 'Bijnor', 'Budaun', 'Bulandshahr', 'Chandauli', 'Chitrakoot', 'Deoria', 'Etah', 'Etawah', 'Farrukhabad', 'Fatehpur', 'Firozabad', 'Gautam Buddha Nagar', 'Ghaziabad', 'Ghazipur', 'Gonda', 'Gorakhpur', 'Hamirpur', 'Hapur', 'Hardoi', 'Hathras', 'Jalaun', 'Jaunpur', 'Jhansi', 'Kannauj', 'Kanpur Dehat', 'Kanpur Nagar', 'Kasganj', 'Kaushambi', 'Kushinagar', 'Lakhimpur Kheri', 'Lalitpur', 'Lucknow', 'Maharajganj', 'Mahoba', 'Mainpuri', 'Mathura', 'Mau', 'Meerut', 'Mirzapur', 'Moradabad', 'Muzaffarnagar', 'Pilibhit', 'Pratapgarh', 'Prayagraj', 'Raebareli', 'Rampur', 'Saharanpur', 'Sambhal', 'Sant Kabir Nagar', 'Shahjahanpur', 'Shamli', 'Shravasti', 'Siddharthnagar', 'Sitapur', 'Sonbhadra', 'Sultanpur', 'Unnao', 'Varanasi'],
    'Uttarakhand': ['Almora', 'Bageshwar', 'Chamoli', 'Champawat', 'Dehradun', 'Haridwar', 'Nainital', 'Pauri Garhwal', 'Pithoragarh', 'Rudraprayag', 'Tehri Garhwal', 'Udham Singh Nagar', 'Uttarkashi'],
    'West Bengal': ['Alipurduar', 'Bankura', 'Birbhum', 'Cooch Behar', 'Dakshin Dinajpur', 'Darjeeling', 'Hooghly', 'Howrah', 'Jalpaiguri', 'Jhargram', 'Kalimpong', 'Kolkata', 'Malda', 'Murshidabad', 'Nadia', 'North 24 Parganas', 'Paschim Bardhaman', 'Paschim Medinipur', 'Purba Bardhaman', 'Purba Medinipur', 'Purulia', 'South 24 Parganas', 'Uttar Dinajpur']
}

# --- MASTER CORRECTION MAP ---
# This dictionary maps known variants (lowercase) to the official names.
# This is a sample; a full one would be built from our entire conversation.
MASTER_CORRECTION_MAP = {
    # Andhra Pradesh / Telangana
    "anantapur": "Ananthapuramu", "cuddapah": "Y.S.R.", "spsr nellore": "S.P.S. Nellore",
    "k.v.rangareddy": "Rangareddy", "ranga reddy": "Rangareddy",
    # Haryana
    "gurgaon": "Gurugram", "mewat": "Nuh",
    # Uttar Pradesh
    "allahabad": "Prayagraj", "faizabad": "Ayodhya", "jyotiba phule nagar": "Amroha",
    # Maharashtra
    "aurangabad": "Chhatrapati Sambhajinagar", "osmanabad": "Dharashiv",
    # Karnataka
    "bangalore": "Bengaluru Urban", "belgaum": "Belagavi", "gulbarga": "Kalaburagi", "mysore": "Mysuru",
    # West Bengal
    "barddhaman": "Purba Bardhaman", "burdwan": "Purba Bardhaman", # Note: Ambiguous, mapping to one for simplicity
    "darjiling": "Darjeeling",
    # Add all other mappings from our conversation here...
}


def clean_dataframe(df, state_col, district_col):
    """Applies the cleaning logic to an entire DataFrame."""
    
    # Create a quick-lookup set of official districts for each state (all lowercase)
    official_districts_lower = defaultdict(set)
    for state, districts in OFFICIAL_DISTRICTS_PER_STATE.items():
        official_districts_lower[state] = {d.lower() for d in districts}

    # --- Cleaning Function to be applied to each row ---
    def get_corrected_district(row):
        state = row[state_col]
        district_original = str(row[district_col]).strip()
        district_lower = district_original.lower()

        # Case 1: The district is in our direct correction map
        if district_lower in MASTER_CORRECTION_MAP:
            return MASTER_CORRECTION_MAP[district_lower]

        # Case 2: The district is already official for that state
        if state in official_districts_lower and district_lower in official_districts_lower[state]:
            return district_original # It's already valid
            
        # Case 3: The district is invalid/unmapped
        return None

    # Apply the function to get a new series of cleaned district names
    cleaned_districts = df.apply(get_corrected_district, axis=1)
    
    # Create a copy to avoid SettingWithCopyWarning
    df_cleaned = df.copy()
    df_cleaned['district'] = cleaned_districts
    
    # Drop rows where the district was invalid (returned None)
    df_cleaned.dropna(subset=['district'], inplace=True)
    
    return df_cleaned


def create_level3_files():
    """Main function to process level-2 files and create cleaned level-3 files."""
    
    # Define source and destination directories
    level2_dir = os.path.join('data', 'level2_processing')
    level3_dir = os.path.join('data', 'level3_processing')

    # Create the level-3 directory
    os.makedirs(level3_dir, exist_ok=True)
    print(f"Directory '{level3_dir}' is ready.")

    files_to_process = [
        'processed_biometric_data.csv',
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv'
    ]

    # Find the state and district column names (assuming they are consistent)
    # You may need to adjust these if column names vary per file
    state_column = 'state'
    district_column = 'district'

    for filename in files_to_process:
        input_path = os.path.join(level2_dir, filename)
        output_path = os.path.join(level3_dir, filename)

        if not os.path.exists(input_path):
            print(f"\n--- Warning: File '{filename}' not found. Skipping. ---")
            continue

        print(f"\n--- Processing '{filename}' ---")
        df = pd.read_csv(input_path)
        
        # Report "Before" count
        before_count = len(df)
        print(f"Before cleaning, row count: {before_count}")

        # Perform the cleaning
        cleaned_df = clean_dataframe(df, state_column, district_column)
        
        # Report "After" count
        after_count = len(cleaned_df)
        print(f"After cleaning, row count is: {after_count}")
        print(f"Rows removed: {before_count - after_count}")

        # Save the cleaned file
        cleaned_df.to_csv(output_path, index=False)
        print(f"Successfully saved cleaned file to '{output_path}'")

    print("\n--- All files processed. Level 3 data is ready. ---")


if __name__ == "__main__":
    create_level3_files()

Directory 'data\level3_processing' is ready.

--- Processing 'processed_biometric_data.csv' ---
Before cleaning, row count: 1861108
After cleaning, row count is: 1571972
Rows removed: 289136
Successfully saved cleaned file to 'data\level3_processing\processed_biometric_data.csv'

--- Processing 'processed_demographic_data.csv' ---
Before cleaning, row count: 2071687
After cleaning, row count is: 1739974
Rows removed: 331713
Successfully saved cleaned file to 'data\level3_processing\processed_demographic_data.csv'

--- Processing 'processed_enrolment_data.csv' ---
Before cleaning, row count: 1006007
After cleaning, row count is: 875852
Rows removed: 130155
Successfully saved cleaned file to 'data\level3_processing\processed_enrolment_data.csv'

--- All files processed. Level 3 data is ready. ---


In [18]:
import pandas as pd
import json
import os

def load_geojson_districts(filepath):
    """Loads the user-provided JSON file as the source of truth."""
    try:
        with open(filepath, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"FATAL ERROR: The required file '{filepath}' was not found.")
        return None
    except json.JSONDecodeError:
        print(f"FATAL ERROR: The file '{filepath}' is not a valid JSON file.")
        return None

def build_correction_map(geojson_map):
    """
    Builds a master map to correct all known variations to match the GeoJSON names.
    This is the "brain" of the operation.
    """
    # Key: messy name (lowercase). Value: Target name (from GeoJSON).
    correction_map = {}
    
    # --- Data from our extensive conversation, mapped to GeoJSON targets ---

    # Name changes where GeoJSON uses the NEW name
    correction_map.update({
        'gurgaon': 'Gurugram', 'mewat': 'Nuh', 'allahabad': 'Prayagraj', 'faizabad': 'Ayodhya',
        'belgaum': 'Belagavi', 'gulgulbarga': 'Kalaburagi', 'mysore': 'Mysuru',
        'shimoga': 'Shivamogga', 'tumkur': 'Tumakuru', 'bijapur': 'Vijayapura',
        'hoshangabad': 'Narmadapuram', # Note: Your JSON has Hoshangabad, this will be handled later
        'nawanshahr': 'Shahid Bhagat Singh Nagar'
    })

    # Name changes where GeoJSON uses the OLD name
    correction_map.update({
        'dharashiv': 'Osmanabad', 'chhatrapati sambhajinagar': 'Aurangabad'
    })

    # Spelling fixes and variations
    correction_map.update({
        'firozpur': 'Ferozepur', 's.a.s. nagar': 'S.A.S. Nagar', 's.a.s nagar(mohali)': 'S.A.S. Nagar',
        'sri muktsar sahib': 'Sri Muktsar Sahib', 'muktsar': 'Sri Muktsar Sahib',
        'darjiling': 'Darjeeling', 'koch bihar': 'Cooch Behar', 'hugli': 'Hooghly',
        'puruliya': 'Purulia', 'medinipur': None, # Ambiguous, must be dropped
        'barddhaman': None, 'burdwan': None, # Ambiguous, must be dropped
        'k.v.rangareddy': 'Ranga Reddy', 'ranga reddy': 'Ranga Reddy', 'visakhapatanam': 'Visakhapatnam',
        'mammit': 'Mamit', 'anugul': 'Angul', 'baleshwar': 'Balasore', 'debargarh': 'Deogarh',
        'jajapur': 'Jajpur', 'khorda': 'Khordha', 'subarnapur': 'Subarnapur',
        'hardwar': 'Haridwar'
    })
    
    # Cross-check the map against the GeoJSON to resolve conflicts
    # Example: Hoshangabad vs Narmadapuram
    if 'Madhya Pradesh' in geojson_map and 'Hoshangabad' in geojson_map['Madhya Pradesh']:
        correction_map['narmadapuram'] = 'Hoshangabad' # Force mapping to what the GeoJSON expects
        correction_map['hoshangabad'] = 'Hoshangabad'
    else:
        correction_map['hoshangabad'] = 'Narmadapuram' # Default to official if not found

    return correction_map

def harmonize_files():
    """Main function to process level-2 files and create harmonized level-3 files."""
    
    data_dir = 'data'
    level2_dir = os.path.join(data_dir, 'level2_processing')
    level3_dir = os.path.join(data_dir, 'level3_processing')
    geojson_path = os.path.join(data_dir, 'state_districts_list.json')

    # 1. Load the "source of truth"
    geojson_districts = load_geojson_districts(geojson_path)
    if geojson_districts is None:
        return

    # Convert lists to sets for fast lookups
    geojson_sets = {state: set(districts) for state, districts in geojson_districts.items()}

    # 2. Build the correction map
    correction_map = build_correction_map(geojson_districts)

    # 3. Create the destination directory
    os.makedirs(level3_dir, exist_ok=True)
    print(f"Directory '{level3_dir}' is ready.")

    files_to_process = [
        'processed_biometric_data.csv',
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv'
    ]
    
    state_col, district_col = 'state', 'district' # Assuming consistent column names

    for filename in files_to_process:
        input_path = os.path.join(level2_dir, filename)
        if not os.path.exists(input_path):
            print(f"\n--- Warning: File '{filename}' not found in {level2_dir}. Skipping. ---")
            continue

        print(f"\n--- Harmonizing '{filename}' ---")
        df = pd.read_csv(input_path)
        
        before_count = len(df)
        print(f"Before harmonization, row count: {before_count}")

        # --- Harmonization Logic ---
        def get_harmonized_district(row):
            state = row[state_col]
            district_original = str(row[district_col]).strip()
            district_lower = district_original.lower()

            # Step A: Apply direct correction if available
            corrected_name = correction_map.get(district_lower, district_original)

            if corrected_name is None: # Explicitly marked for dropping (e.g., 'Burdwan')
                return None
            
            # Step B: Validate the name against the GeoJSON list for that state
            if state in geojson_sets and corrected_name in geojson_sets[state]:
                return corrected_name # Success!
            
            # If not found, it's invalid
            return None

        # Apply the logic to the DataFrame
        df_copy = df.copy()
        df_copy[district_col] = df_copy.apply(get_harmonized_district, axis=1)
        
        # Drop rows that were not successfully harmonized
        df_copy.dropna(subset=[district_col], inplace=True)
        
        after_count = len(df_copy)
        print(f"After harmonization, row count is: {after_count}")
        print(f"Rows removed: {before_count - after_count}")

        # Save the harmonized file
        output_path = os.path.join(level3_dir, filename)
        df_copy.to_csv(output_path, index=False)
        print(f"Successfully saved harmonized file to '{output_path}'")

    print("\n--- All files harmonized. Level 3 data is ready for plotting. ---")

if __name__ == "__main__":
    harmonize_files()

Directory 'data\level3_processing' is ready.

--- Harmonizing 'processed_biometric_data.csv' ---
Before harmonization, row count: 1861108
After harmonization, row count is: 1521410
Rows removed: 339698
Successfully saved harmonized file to 'data\level3_processing\processed_biometric_data.csv'

--- Harmonizing 'processed_demographic_data.csv' ---
Before harmonization, row count: 2071687
After harmonization, row count is: 1669920
Rows removed: 401767
Successfully saved harmonized file to 'data\level3_processing\processed_demographic_data.csv'

--- Harmonizing 'processed_enrolment_data.csv' ---
Before harmonization, row count: 1006007
After harmonization, row count is: 850291
Rows removed: 155716
Successfully saved harmonized file to 'data\level3_processing\processed_enrolment_data.csv'

--- All files harmonized. Level 3 data is ready for plotting. ---


In [27]:
import json

# If your data is in a file called 'data.json'
with open('in.json', 'r') as f:
    data = json.load(f)

# Extract all "name" values from the properties of each feature
names = [feature['properties']['name'] for feature in data['features']]

# Print the list of names
print(len(names))
for name in names:
    print(name)
names

36
Andaman and Nicobar
Telangana
Andhra Pradesh
Arunachal Pradesh
Assam
Bihar
Chandigarh
Chhattisgarh
Dādra and Nagar Haveli and Damān and Diu
Delhi
Goa
Gujarat
Haryana
Himachal Pradesh
Jharkhand
Karnataka
Kerala
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Nagaland
Orissa
Puducherry
Punjab
Rajasthan
Sikkim
Tamil Nadu
Tripura
Uttar Pradesh
Uttaranchal
West Bengal
Lakshadweep
Jammu and Kashmir
Ladakh


['Andaman and Nicobar',
 'Telangana',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dādra and Nagar Haveli and Damān and Diu',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Orissa',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Tripura',
 'Uttar Pradesh',
 'Uttaranchal',
 'West Bengal',
 'Lakshadweep',
 'Jammu and Kashmir',
 'Ladakh']

In [2]:
import pandas as pd
import numpy as np
import os
import pgeocode

def create_level4_data():
    print("🔹 Starting Level 4 Spatial Cleaning Protocol...")
    
    # 1. Setup Directories
    input_dir = 'data/level3_processing'
    output_dir = 'data/level4_processing'
    if not os.path.exists(output_dir): os.makedirs(output_dir)
    
    files = [
        'processed_biometric_data.csv',
        'processed_demographic_data.csv',
        'processed_enrolment_data.csv'
    ]
    
    # 2. Initialize Geocoder
    nomi = pgeocode.Nominatim('in')
    
    # 3. Process Files
    for filename in files:
        input_path = os.path.join(input_dir, filename)
        if not os.path.exists(input_path):
            print(f"⚠️ Skipped {filename} (Not found)")
            continue
            
        print(f"\nProcessing {filename}...")
        df = pd.read_csv(input_path)
        before_count = len(df)
        
        # --- A. GEOCODE UNIQUE PINCODES ---
        # We only geocode unique pincodes to save time
        unique_pins = df['pincode'].unique().astype(str)
        geo_data = nomi.query_postal_code(unique_pins)
        
        # Create a lookup dataframe
        pin_map = pd.DataFrame({
            'pincode': unique_pins,
            'lat': geo_data.latitude,
            'lon': geo_data.longitude
        })
        
        # Merge Lat/Lon into main dataframe
        df['pincode'] = df['pincode'].astype(str)
        df_merged = df.merge(pin_map, on='pincode', how='left')
        
        # --- B. REMOVE INVALID PINCODES (No Lat/Lon) ---
        df_clean = df_merged.dropna(subset=['lat', 'lon'])
        invalid_count = before_count - len(df_clean)
        
        # --- C. SPATIAL OUTLIER DETECTION (The "100km" Fix) ---
        # For each district, calculate the center. Remove points > 2 Std Devs away.
        def filter_spatial_outliers(group):
            if len(group) < 5: return group # Too small to filter
            
            lat_mean, lat_std = group['lat'].mean(), group['lat'].std()
            lon_mean, lon_std = group['lon'].mean(), group['lon'].std()
            
            # If std is tiny (all points in one spot), keep them
            if lat_std < 0.05 or lon_std < 0.05: return group
            
            # Keep points within 2 Sigma (95% confidence)
            # This deletes the "Mumbai pincode in Delhi" errors
            mask = (np.abs(group['lat'] - lat_mean) <= 2 * lat_std) & \
                   (np.abs(group['lon'] - lon_mean) <= 2 * lon_std)
            return group[mask]

        print("   -> Calculating spatial clusters...")
        # Apply filter per district
        df_final = df_clean.groupby(['state', 'district'], group_keys=False).apply(filter_spatial_outliers)
        
        # --- D. SAVE ---
        after_count = len(df_final)
        outlier_count = len(df_clean) - after_count
        
        # Drop the temp lat/lon columns before saving to keep structure clean
        df_final = df_final.drop(columns=['lat', 'lon'])
        
        output_path = os.path.join(output_dir, filename)
        df_final.to_csv(output_path, index=False)
        
        print(f"   ✅ Cleaned.")
        print(f"   - Original Rows: {before_count:,}")
        print(f"   - Invalid Pincodes Removed: {invalid_count:,}")
        print(f"   - Spatial Outliers Removed: {outlier_count:,}")
        print(f"   - Final Level 4 Rows: {after_count:,}")

if __name__ == "__main__":
    create_level4_data()

c:\Users\haaaa\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


🔹 Starting Level 4 Spatial Cleaning Protocol...

Processing processed_biometric_data.csv...
   -> Calculating spatial clusters...


C:\Users\haaaa\AppData\Local\Temp\ipykernel_19584\1796164387.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_clean.groupby(['state', 'district'], group_keys=False).apply(filter_spatial_outliers)


   ✅ Cleaned.
   - Original Rows: 1,571,972
   - Invalid Pincodes Removed: 40,001
   - Spatial Outliers Removed: 134,488
   - Final Level 4 Rows: 1,397,483

Processing processed_demographic_data.csv...
   -> Calculating spatial clusters...


C:\Users\haaaa\AppData\Local\Temp\ipykernel_19584\1796164387.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_clean.groupby(['state', 'district'], group_keys=False).apply(filter_spatial_outliers)


   ✅ Cleaned.
   - Original Rows: 1,739,974
   - Invalid Pincodes Removed: 46,691
   - Spatial Outliers Removed: 149,695
   - Final Level 4 Rows: 1,543,588

Processing processed_enrolment_data.csv...
   -> Calculating spatial clusters...


C:\Users\haaaa\AppData\Local\Temp\ipykernel_19584\1796164387.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_final = df_clean.groupby(['state', 'district'], group_keys=False).apply(filter_spatial_outliers)


   ✅ Cleaned.
   - Original Rows: 875,852
   - Invalid Pincodes Removed: 18,264
   - Spatial Outliers Removed: 74,765
   - Final Level 4 Rows: 782,823


In [ ]:
import json

# If your data is in a file called 'data.json'
with open('in.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract all "name" values from the properties of each feature
names = [feature['properties']['name'] for feature in data['features']]

# Print the list of names
print(len(names))
for name in names:
    print(name)
print(names)